# Gradio & HuggingFace Transformers Tutorial

A comprehensive tutorial on building interactive machine learning applications using Gradio and HuggingFace Transformers, designed for Google Colab.

## Table of Contents

- [Overview](#overview)
- [Setup Instructions](#setup-instructions)
- [Part 1: Gradio Basics](#part-1-gradio-basics)
- [Part 2: HuggingFace Integration](#part-2-huggingface-integration)
- [Key Concepts](#key-concepts)
- [Troubleshooting](#troubleshooting)
- [Resources](#resources)

## Overview

This tutorial covers:

1. **Gradio Fundamentals**: Learn to build interactive UIs with rows, columns, images, input fields, and sliders
2. **Creative Application**: Build a "Mood-Based Color Palette Generator" that creates color schemes based on user preferences
3. **HuggingFace Integration**: Develop an "AI Creative Studio" combining sentiment analysis, image captioning, and text generation

## Setup Instructions

### Running in Google Colab

1. Open a new Colab notebook: [colab.research.google.com](https://colab.research.google.com)
2. Install required libraries:

```python
!pip install gradio transformers torch pillow
```

3. Copy and run the code cells from this tutorial
4. Click the generated public URL to access your app

### Local Installation (Optional)

```bash
pip install gradio transformers torch pillow
```

## Part 1: Gradio Basics

### What is Gradio?

Gradio is a Python library that enables you to quickly create web interfaces for machine learning models or any Python function. It's particularly popular for:

- Rapid prototyping of ML demos
- Creating shareable interfaces without web development knowledge
- Integration with HuggingFace Spaces for deployment

### Core Components

**Layout Components:**
- `gr.Blocks()`: Main container for custom layouts
- `gr.Row()`: Horizontal layout
- `gr.Column()`: Vertical layout
- `gr.Tab()`: Tabbed interface

**Input/Output Components:**
- `gr.Textbox()`: Text input/output
- `gr.Slider()`: Numeric slider
- `gr.Image()`: Image upload/display
- `gr.Dropdown()`: Selection menu
- `gr.Button()`: Clickable button
- `gr.Gallery()`: Image grid display

### Example: Mood-Based Color Palette Generator

This creative application generates harmonious color palettes based on:
- User's mood selection
- Temperature preference (warm/cool)
- Saturation level
- Number of colors

The app demonstrates:
- Complex layouts with rows and columns
- Multiple input types (dropdown, slider, radio)
- Dynamic image generation
- Real-time updates

**Key Features:**
- Predefined mood-to-color mappings
- HSV color space manipulation for harmony
- PIL-based palette visualization
- Responsive design with organized sections

## Part 2: HuggingFace Integration

### What is HuggingFace Transformers?

HuggingFace Transformers is a library providing pre-trained models for:
- Natural Language Processing (NLP)
- Computer Vision
- Audio Processing
- Multimodal tasks

### Pipelines API

The `pipeline()` function provides simple access to models:

```python
from transformers import pipeline

# Sentiment Analysis
sentiment = pipeline("sentiment-analysis")
result = sentiment("I love this tutorial!")

# Image Captioning
captioner = pipeline("image-to-text")
caption = captioner("image.jpg")

# Text Generation
generator = pipeline("text-generation")
text = generator("Once upon a time")
```

### Example: AI Creative Studio

This application combines three AI capabilities:

1. **Sentiment Analyzer**: Analyzes emotional tone of text
2. **Image Storyteller**: Generates captions and creative stories from images
3. **Creative Writer**: Generates stories from prompts

**Models Used:**
- `distilbert-base-uncased-finetuned-sst-2-english`: Fast sentiment analysis
- `Salesforce/blip-image-captioning-base`: Image understanding
- `gpt2`: Text generation

**Technical Highlights:**
- Multiple pipeline integration
- Error handling for model loading
- Tabbed interface for organization
- Image upload and processing
- Adjustable generation parameters

## Key Concepts

### Gradio Interface Patterns

**Simple Interface:**
```python
gr.Interface(fn=function, inputs=..., outputs=...)
```

**Custom Blocks:**
```python
with gr.Blocks() as demo:
    with gr.Row():
        input_col = gr.Column()
        output_col = gr.Column()
```

### Event Handling

Connect components with `.click()`, `.change()`, `.submit()`:

```python
button.click(fn=process, inputs=[input1, input2], outputs=output)
```

### Best Practices

1. **Error Handling**: Wrap model calls in try-except blocks
2. **Loading States**: Show progress for long operations
3. **Clear Descriptions**: Use `label` and `info` parameters
4. **Responsive Design**: Use rows/columns for proper layout
5. **Resource Management**: Load heavy models once, not per request

## Troubleshooting

### Common Issues

**Issue**: ModuleNotFoundError
**Solution**: Ensure all packages are installed: `!pip install gradio transformers torch pillow`

**Issue**: CUDA out of memory
**Solution**: Use CPU by not specifying device, or use smaller models

**Issue**: Model download slow
**Solution**: Models cache automatically. First run takes time; subsequent runs are faster

**Issue**: Gradio tunnel not working
**Solution**: Try `demo.launch(share=True, debug=True)` for more information

### Memory Optimization

For Colab's free tier:
- Use smaller models (distilbert instead of bert)
- Set `low_cpu_mem_usage=True` when loading models
- Clear outputs regularly: `Runtime > Manage Sessions`

## Resources

### Official Documentation
- [Gradio Docs](https://www.gradio.app/docs)
- [HuggingFace Transformers](https://huggingface.co/docs/transformers)
- [HuggingFace Models](https://huggingface.co/models)

### Useful Links
- [Gradio Guides](https://www.gradio.app/guides)
- [Pipeline Task Documentation](https://huggingface.co/docs/transformers/task_summary)
- [Gradio on HuggingFace Spaces](https://huggingface.co/spaces)

### Further Learning
- Explore more models at [HuggingFace Hub](https://huggingface.co/models)
- Join [HuggingFace Discord](https://discord.com/invite/hugging-face)
- Check [Gradio Community](https://github.com/gradio-app/gradio/discussions)

## Learning Outcomes

After completing this tutorial, you will be able to:

Build custom Gradio interfaces with complex layouts
Integrate multiple input and output components
Use HuggingFace pipelines for various AI tasks
Combine multiple models in a single application
Deploy interactive demos in Google Colab
Handle images and text with AI models
Create shareable ML applications


In [ ]:
!pip install -q gradio transformers torch pillow

print("Installation complete! Ready to build apps.")

Installation complete! Ready to build apps.


# We'll build a "Mood-Based Color Palette Generator"


In [ ]:
import gradio as gr
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import colorsys

In [ ]:
def generate_color_palette(mood, temperature, saturation, num_colors):
    """
    Generates a harmonious color palette based on mood and preferences.

    Args:
        mood: Base emotion (happy, calm, energetic, melancholic, creative)
        temperature: Warm or cool colors
        saturation: Color intensity (0-100)
        num_colors: Number of colors to generate (3-8)

    Returns:
        PIL Image of the color palette with hex codes
    """

    # Define base hues for each mood (0-360 degrees)
    mood_hues = {
        "Happy": 45,      # Yellow-Orange
        "Calm": 200,      # Blue
        "Energetic": 0,   # Red
        "Melancholic": 240,  # Purple
        "Creative": 280   # Magenta
    }

    # Get base hue
    base_hue = mood_hues[mood]

    # Adjust for temperature
    if temperature == "Warm":
        base_hue = (base_hue - 30) % 360  # Shift toward red-yellow
    else:  # Cool
        base_hue = (base_hue + 30) % 360  # Shift toward blue-green

    # Convert saturation from 0-100 to 0-1
    sat = saturation / 100.0

    # Generate harmonious colors using different strategies
    colors = []
    for i in range(num_colors):
        # Create variation using analogous colors (nearby hues)
        hue_variation = (base_hue + (i * 30)) % 360

        # Slight saturation and value variations for interest
        s = max(0.3, min(1.0, sat + (i % 2) * 0.1))
        v = 0.85 + (i % 3) * 0.05

        # Convert HSV to RGB
        r, g, b = colorsys.hsv_to_rgb(hue_variation / 360.0, s, v)
        colors.append((int(r * 255), int(g * 255), int(b * 255)))

    # Create image to display palette
    palette_width = 600
    palette_height = 400
    swatch_height = palette_height // 2

    img = Image.new('RGB', (palette_width, palette_height), 'white')
    draw = ImageDraw.Draw(img)

    # Draw color swatches
    swatch_width = palette_width // num_colors
    for i, color in enumerate(colors):
        x0 = i * swatch_width
        x1 = (i + 1) * swatch_width
        draw.rectangle([x0, 0, x1, swatch_height], fill=color)

        # Add hex code labels
        hex_code = '#{:02x}{:02x}{:02x}'.format(*color)

        # Calculate text position
        text_bbox = draw.textbbox((0, 0), hex_code)
        text_width = text_bbox[2] - text_bbox[0]
        text_x = x0 + (swatch_width - text_width) // 2
        text_y = swatch_height + 20

        # Draw text with background for readability
        draw.rectangle([text_x - 5, text_y - 5, text_x + text_width + 5, text_y + 20],
                      fill='white')
        draw.text((text_x, text_y), hex_code, fill='black')

        # Draw RGB values
        rgb_text = f"RGB({color[0]}, {color[1]}, {color[2]})"
        rgb_bbox = draw.textbbox((0, 0), rgb_text)
        rgb_width = rgb_bbox[2] - rgb_bbox[0]
        rgb_x = x0 + (swatch_width - rgb_width) // 2
        rgb_y = text_y + 30

        draw.text((rgb_x, rgb_y), rgb_text, fill='gray')

    # Add title
    title = f"{mood} {temperature} Palette"
    title_bbox = draw.textbbox((0, 0), title)
    title_width = title_bbox[2] - title_bbox[0]
    draw.text((palette_width // 2 - title_width // 2, swatch_height + 80),
             title, fill='black')

    return img

In [ ]:
# Build the Gradio interface with custom layout
with gr.Blocks(title="Mood Color Palette Generator", theme=gr.themes.Soft()) as demo_basic:

    # Header
    gr.Markdown("""
    # 🎨 Mood-Based Color Palette Generator
    Create beautiful, harmonious color palettes based on your mood and preferences!
    """)

    # Main layout with two columns
    with gr.Row():
        # Left column: Inputs
        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Palette Settings")

            mood_input = gr.Dropdown(
                choices=["Happy", "Calm", "Energetic", "Melancholic", "Creative"],
                value="Happy",
                label="Choose Your Mood",
                info="Select the emotion you want to express"
            )

            temperature_input = gr.Radio(
                choices=["Warm", "Cool"],
                value="Warm",
                label="Color Temperature",
                info="Warm colors (reds, oranges) vs Cool colors (blues, greens)"
            )

            saturation_input = gr.Slider(
                minimum=0,
                maximum=100,
                value=70,
                step=5,
                label="Saturation Level",
                info="How intense should the colors be? (0=gray, 100=vivid)"
            )

            num_colors_input = gr.Slider(
                minimum=3,
                maximum=8,
                value=5,
                step=1,
                label="Number of Colors",
                info="How many colors in your palette?"
            )

            generate_btn = gr.Button("🎨 Generate Palette", variant="primary", size="lg")

        # Right column: Output
        with gr.Column(scale=2):
            gr.Markdown("### 🖼️ Your Color Palette")
            output_image = gr.Image(
                label="Generated Palette",
                type="pil",
                height=400
            )

            gr.Markdown("""
            **💡 Tips:**
            - Use warm colors for energy and excitement
            - Cool colors create calm and professional vibes
            - Lower saturation for subtle, elegant designs
            - Higher saturation for bold, eye-catching visuals
            """)

    # Examples section
    gr.Markdown("### 🌟 Try These Presets")
    gr.Examples(
        examples=[
            ["Happy", "Warm", 80, 5],
            ["Calm", "Cool", 50, 4],
            ["Energetic", "Warm", 95, 6],
            ["Melancholic", "Cool", 40, 4],
            ["Creative", "Warm", 75, 7],
        ],
        inputs=[mood_input, temperature_input, saturation_input, num_colors_input],
        outputs=output_image,
        fn=generate_color_palette,
        cache_examples=False,
    )

    # Connect button to function
    generate_btn.click(
        fn=generate_color_palette,
        inputs=[mood_input, temperature_input, saturation_input, num_colors_input],
        outputs=output_image
    )

    # Also update on any input change for live preview
    for input_component in [mood_input, temperature_input, saturation_input, num_colors_input]:
        input_component.change(
            fn=generate_color_palette,
            inputs=[mood_input, temperature_input, saturation_input, num_colors_input],
            outputs=output_image
        )

/tmp/ipython-input-575575056.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="Mood Color Palette Generator", theme=gr.themes.Soft()) as demo_basic:


In [ ]:
print("\n" + "="*70)
print("PART 1: GRADIO BASICS - Launching Color Palette Generator...")
print("="*70 + "\n")

demo_basic.launch(share=True, debug=False)


PART 1: GRADIO BASICS - Launching Color Palette Generator...

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://aa66616dd437b3ae5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# ============================================================================
# PART 2: HUGGINGFACE TRANSFORMERS INTEGRATION
# ============================================================================
# We'll build an "AI Creative Studio" with multiple AI models
# ============================================================================

from transformers import pipeline
import torch

print("\n" + "="*70)
print("PART 2: Loading HuggingFace Models...")
print("="*70 + "\n")

# Check if CUDA is available
device = 0 if torch.cuda.is_available() else -1
print(f"Using device: {'GPU (CUDA)' if device == 0 else 'CPU'}")


PART 2: Loading HuggingFace Models...

Using device: GPU (CUDA)


In [ ]:
# Load models (this may take a minute on first run)
print("\n⏳ Loading AI models... (this may take 1-2 minutes)")

try:
    # Sentiment Analysis
    print("  Loading sentiment analyzer...")
    sentiment_analyzer = pipeline(
        "sentiment-analysis",
        model="distilbert-base-uncased-finetuned-sst-2-english",
        device=device
    )
    print("  Sentiment analyzer loaded")

    # Image Captioning
    print("  Loading image captioner...")
    image_captioner = pipeline(
        "image-to-text",
        model="Salesforce/blip-image-captioning-base",
        device=device
    )
    print("  Image captioner loaded")

    # Text Generation
    print("  Loading text generator...")
    text_generator = pipeline(
        "text-generation",
        model="gpt2",
        device=device
    )
    print("  Text generator loaded")

    print("\n All models loaded successfully!\n")

except Exception as e:
    print(f"\n Error loading models: {e}")
    print(" Try restarting runtime if you encounter memory issues\n")


⏳ Loading AI models... (this may take 1-2 minutes)
  Loading sentiment analyzer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


  Text generator loaded

 All models loaded successfully!



In [ ]:
# Function definitions for each AI feature

def analyze_sentiment(text):
    """Analyze sentiment of input text"""
    if not text or text.strip() == "":
        return "Please enter some text to analyze.", None

    try:
        result = sentiment_analyzer(text)[0]
        label = result['label']
        score = result['score']

        # Create a visual representation
        sentiment_emoji = "😊" if label == "POSITIVE" else "😔"
        confidence = f"{score * 100:.1f}%"

        output = f"""
### Analysis Results {sentiment_emoji}

**Sentiment:** {label}
**Confidence:** {confidence}

The text expresses a **{label.lower()}** sentiment with {confidence} confidence.
        """

        return output, result
    except Exception as e:
        return f"Error: {str(e)}", None

def caption_and_story(image, creativity):
    """Generate caption and creative story from image"""
    if image is None:
        return "Please upload an image.", ""

    try:
        # Generate caption
        caption_result = image_captioner(image)[0]['generated_text']

        # Generate creative story based on the caption
        story_prompt = f"Write a short creative story about: {caption_result}"

        story_result = text_generator(
            story_prompt,
            max_length=150,
            num_return_sequences=1,
            temperature=creativity,
            do_sample=True,
            top_k=50,
            top_p=0.95
        )[0]['generated_text']

        # Clean up the story
        story = story_result.replace(story_prompt, "").strip()

        return f"**Caption:** {caption_result}", story
    except Exception as e:
        return f"Error: {str(e)}", ""

def generate_creative_text(prompt, length, creativity):
    """Generate creative text from prompt"""
    if not prompt or prompt.strip() == "":
        return "Please enter a prompt to generate text."

    try:
        # Calculate max_length based on user input
        max_length = min(length, 200)  # Cap at 200 tokens

        result = text_generator(
            prompt,
            max_length=max_length,
            num_return_sequences=1,
            temperature=creativity,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            pad_token_id=text_generator.tokenizer.eos_token_id
        )[0]['generated_text']

        return result
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Build the AI Creative Studio interface
with gr.Blocks(title="AI Creative Studio", theme=gr.themes.Base()) as demo_ai:

    # Header
    gr.Markdown("""
    # 🤖 AI Creative Studio
    ### Powered by HuggingFace Transformers

    Explore the capabilities of state-of-the-art AI models for sentiment analysis,
    image understanding, and creative writing!
    """)

    # Tabs for different AI features
    with gr.Tabs():

        # Tab 1: Sentiment Analysis
        with gr.Tab(" Sentiment Analyzer"):
            gr.Markdown("""
            ### Analyze the emotional tone of any text
            This uses a DistilBERT model fine-tuned for sentiment classification.
            """)

            with gr.Row():
                with gr.Column():
                    sentiment_input = gr.Textbox(
                        label="Enter Text to Analyze",
                        placeholder="Type or paste any text here...",
                        lines=5
                    )
                    sentiment_btn = gr.Button("Analyze Sentiment", variant="primary")

                    gr.Examples(
                        examples=[
                            "I absolutely love this tutorial! It's so helpful and well-explained.",
                            "This is the worst experience I've ever had. Very disappointed.",
                            "The weather today is okay, nothing special.",
                            "I'm excited about learning AI and building cool projects!",
                        ],
                        inputs=sentiment_input,
                    )

                with gr.Column():
                    sentiment_output = gr.Markdown(label="Analysis Results")

            sentiment_btn.click(
                fn=analyze_sentiment,
                inputs=sentiment_input,
                outputs=[sentiment_output, gr.JSON(visible=False)]
            )

        # Tab 2: Image Storyteller
        with gr.Tab("Image Storyteller"):
            gr.Markdown("""
            ### Transform images into captions and creative stories
            Upload an image and watch AI describe it and create a story!
            """)

            with gr.Row():
                with gr.Column():
                    image_input = gr.Image(
                        label="Upload an Image",
                        type="pil",
                        height=300
                    )

                    creativity_slider = gr.Slider(
                        minimum=0.5,
                        maximum=1.5,
                        value=1.0,
                        step=0.1,
                        label="Creativity Level",
                        info="Higher = more creative/unexpected stories"
                    )

                    caption_btn = gr.Button("✨ Generate Caption & Story", variant="primary")

                with gr.Column():
                    caption_output = gr.Markdown(label="Image Caption")
                    story_output = gr.Textbox(
                        label="Creative Story",
                        lines=8,
                        interactive=False
                    )

            caption_btn.click(
                fn=caption_and_story,
                inputs=[image_input, creativity_slider],
                outputs=[caption_output, story_output]
            )

        # Tab 3: Creative Writer
        with gr.Tab("Creative Writer"):
            gr.Markdown("""
            ### Generate creative text from your prompts
            Start a story, continue a narrative, or explore creative ideas!
            """)

            with gr.Row():
                with gr.Column():
                    prompt_input = gr.Textbox(
                        label="Enter Your Prompt",
                        placeholder="Once upon a time in a distant galaxy...",
                        lines=3
                    )

                    with gr.Row():
                        length_slider = gr.Slider(
                            minimum=50,
                            maximum=200,
                            value=100,
                            step=10,
                            label="Length (tokens)",
                            info="Longer = more text generated"
                        )

                        creativity_slider2 = gr.Slider(
                            minimum=0.5,
                            maximum=1.5,
                            value=0.9,
                            step=0.1,
                            label="Creativity",
                            info="Higher = more surprising outputs"
                        )

                    generate_btn = gr.Button("Generate Text", variant="primary")

                    gr.Examples(
                        examples=[
                            "In a world where AI and humans work together,",
                            "The old lighthouse keeper discovered a mysterious message",
                            "She opened the ancient book and found",
                            "The robot looked at its reflection and wondered",
                        ],
                        inputs=prompt_input,
                    )

                with gr.Column():
                    text_output = gr.Textbox(
                        label="Generated Text",
                        lines=12,
                        interactive=False
                    )

            generate_btn.click(
                fn=generate_creative_text,
                inputs=[prompt_input, length_slider, creativity_slider2],
                outputs=text_output
            )

    # Footer with model information
    gr.Markdown("""
    ---
    ### 🔧 Models Used

    - **Sentiment Analysis**: `distilbert-base-uncased-finetuned-sst-2-english`
    - **Image Captioning**: `Salesforce/blip-image-captioning-base`
    - **Text Generation**: `gpt2`

    All models are from the [HuggingFace Model Hub](https://huggingface.co/models).
    """)


/tmp/ipython-input-199309902.py:2: DeprecationWarning: The 'theme' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'theme' to Blocks.launch() instead.
  with gr.Blocks(title="AI Creative Studio", theme=gr.themes.Base()) as demo_ai:


In [ ]:
#Launch the AI Studio
print("\n" + "="*70)
print("PART 2: Launching AI Creative Studio...")
print("="*70 + "\n")

demo_ai.launch(share=True, debug=False)


PART 2: Launching AI Creative Studio...

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e09fc2fe2fd1fbd79d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


TUTORIAL COMPLETE!

Congratulations! You've built two complete Gradio applications:

1️⃣  Mood-Based Color Palette Generator
   - Custom layouts with rows and columns
   - Multiple input types (dropdown, radio, sliders)
   - Dynamic image generation
   
2️⃣  AI Creative Studio
   - Sentiment analysis with DistilBERT
   - Image captioning with BLIP
   - Text generation with GPT-2
   - Tabbed interface organization

 Key Takeaways:
   ✓ Gradio makes it easy to build ML interfaces
   ✓ HuggingFace pipelines provide simple access to powerful models
   ✓ You can combine multiple models in one application
   ✓ share=True gives you a public URL to share your app

 Next Steps:
   • Explore more models at huggingface.co/models
   • Try different pipeline tasks (translation, summarization, etc.)
   • Deploy to HuggingFace Spaces for permanent hosting
   • Experiment with model parameters for different results

